<a href="https://colab.research.google.com/github/saba6099/Deep-Learning-for-vision/blob/master/CIFAR_Classification_with_basic_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import random as rand
%matplotlib inline
import matplotlib.pyplot as plt 
from torch.autograd import Variable

In [0]:
transform = transforms.Compose([transforms.ToTensor()])

# transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))


In [0]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

0it [00:00, ?it/s]

 99%|█████████▉| 168747008/170498071 [00:12<00:00, 16984701.55it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
CLASSES[trainset[100][1]]

'ship'

In [0]:
trainset[1][0].size()

torch.Size([3, 32, 32])

In [0]:
'''
STEP 2: MAKING DATASET ITERABLE
'''
 
batch_size = 100
n_iters = 50000
num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)
print("length of trainset:", len(trainset))
print("Number of epochs: ", num_epochs)
train_loader = torch.utils.data.DataLoader(dataset=trainset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=testset, 
                                          batch_size=batch_size, 
                                          shuffle=False)
 

length of trainset: 50000
Number of epochs:  100


In [0]:
batch_size

100

In [0]:
import torch.nn as nn

'''
STEP 3: CREATE MODEL CLASS
'''
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        # Linear function 1: 784 --> 100
        self.fc1 = nn.Linear(input_dim, 500) 
        # Non-linearity 1
        self.relu = nn.ReLU()
        
        # Linear function 2: 100 --> 100
        self.fc2 = nn.Linear(500, 10)
    
    
    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        # Non-linearity 1
        out = self.relu(out)
        
        # Linear function 2
        out = self.fc2(out)
        return out
 

In [0]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 32*32*3
hidden_dim = 1500
output_dim = 10
 
model = LogisticRegressionModel(input_dim,hidden_dim, output_dim)

In [0]:
use_cuda = True
import matplotlib.pyplot as plt

In [0]:
if torch.cuda.is_available():
    avDev = torch.device("cuda")
else:
    avDev = torch.device("cpu")

print(avDev)

cuda


In [0]:
#  USE GPU FOR MODEL  #
#######################
 
model.to(avDev)
 
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss().to(avDev)

 
'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.01
 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum= 0.9)


In [0]:
# Training curve
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
train_summary = SummaryWriter('train_summary')

In [0]:
'''
STEP 7: TRAIN THE MODEL
'''
loss_values = []
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
         
        #print(images.size()) #torch.Size([100, 1, 28, 28])
        #print(images.view(-1, 32*32*3).size()) #torch.Size([100, 784])
        images = images.view(-1, 32*32*3).to(avDev)
        labels = labels.to(avDev)
        #print(labels) 
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)#
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
         
        iter += 1
         
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                images = images.view(-1, 32*32*3).to(avDev)
                 
                # Forward pass only to get logits/output
                outputs = model(images)
                 
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                correct += (predicted.cpu() == labels.cpu()).sum().float()
             
            accuracy = 100. * correct / total
            loss_values.append(loss.item())
            # loss = np.array(train_summary.read_scalar("Loss"))
            # plt.plot(loss[:, 0], loss[:, 1], label='train loss')
            # plt.legend()
            # plt.show()
            # plt.plot(loss.item(), iter)
            # print("Loss", type(loss.item()))
            # plt.show()
             
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))
plt.plot(loss_values, 'r')
plt.show()

Iteration: 500. Loss: 1.6001091003417969. Accuracy: 42.290000915527344
Iteration: 1000. Loss: 1.2527391910552979. Accuracy: 44.38999938964844
Iteration: 1500. Loss: 1.265075922012329. Accuracy: 46.099998474121094
Iteration: 2000. Loss: 1.309683084487915. Accuracy: 46.04999923706055
Iteration: 2500. Loss: 1.131248116493225. Accuracy: 46.81999969482422
Iteration: 3000. Loss: 1.1907014846801758. Accuracy: 46.65999984741211
Iteration: 3500. Loss: 1.182090163230896. Accuracy: 47.7400016784668
Iteration: 4000. Loss: 0.9841108918190002. Accuracy: 47.470001220703125
Iteration: 4500. Loss: 1.2116827964782715. Accuracy: 48.75
Iteration: 5000. Loss: 1.1654516458511353. Accuracy: 49.97999954223633
Iteration: 5500. Loss: 0.9077140688896179. Accuracy: 50.68000030517578
Iteration: 6000. Loss: 1.0041264295578003. Accuracy: 48.869998931884766
Iteration: 6500. Loss: 1.0187968015670776. Accuracy: 50.0099983215332
Iteration: 7000. Loss: 0.8721417784690857. Accuracy: 51.33000183105469
Iteration: 7500. Loss

Getting Maximum Accuracy of 49 percent.
Things to do:
1) learning curve
2) Confusion Matrix